In [ ]:
def print():
    print("a")
    return

In [17]:
"""
データベースアクセスを行う
"""
import psycopg2

DBNAME = "hiura_db"
USER = "postgres"
PASS = "usadasql"
URL = "localhost"
PORT = 5434

#矩形範囲の道路データを取得する
#(y1, x1):左上の緯度経度 (y2, x2):右下の緯度経度 n:範囲を拡大する倍率
#以下お試しのためにデフォルト値を入力している
n = 1
y1=35.16398550407737 
x1=136.9278430938721
y2=35.16223130530258
x2=136.93067550659183
x=0.0028324127197265625
y=0.0017541987747833332
'''
x = abs(x2-x1)
y = abs(y2-y1)
x1 = x1-x*(n-1)
x2 = x2+x*(n-1)
y1 = y1+y*(n-1)
y2 = y2-y*(n-1)'
'''

conn = psycopg2.connect(database=DBNAME, user=USER, password=PASS, host=URL, port=PORT)
cur = conn.cursor()


try:
    statement = "select  "\
                    "id, osm_name, osm_source_id, osm_target_id, clazz, source, target, km, cost, x1, y1, x2, y2, geom_way  "\
                "from "\
                    "osm_japan_car_2po_4pgr  "\
                "where "\
                    "st_intersects("\
                        "st_geomFromText("\
                            "'polygon(("+str(x1)+" "+str(y2)+","+str(x2)+" "+str(y2)+","+str(x2)+" "+str(y1)+","+str(x1)+" "+str(y1)+","+str(x1)+" "+str(y2)+"))', 4326),"\
                        "geom_way) "\
                    ""
    cur.execute(statement)
    docs = cur.fetchall()

    link = []
    
    length = []
    for rs in docs:
        link.append([[float(rs[10]), float(rs[9])], [float(rs[12]), float(rs[11])]])
        length.append(float(rs[7]))
        print("rs：",rs)
    print("link:",link)

except Exception as e:
    print(e)


print("link:",link)

#ストロークデータを獲得したい
# ①のクエリで取得したリンクIDリスト
link_ids_in_area = [int(rs[0]) for rs in docs]

if not link_ids_in_area:
    print("選択範囲に該当するリンクがありません")
else:
    id_array_str = ','.join(map(str, link_ids_in_area))
    sql = f"""
    SELECT id, link_ids, stroke_length, stroke_clazz, ST_AsText(arc_series) AS wkt
    FROM stroke_v2.stroke_table
    WHERE EXISTS (
        SELECT 1 FROM unnest(string_to_array(link_ids, ',')) AS lid
        WHERE lid::int = ANY(ARRAY[{id_array_str}])
    )
    """
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        id, link_ids, stroke_length, stroke_clazz, wkt = row
        print(f"id:{id} links:{link_ids} length:{stroke_length} clazz:{stroke_clazz} wkt:{wkt}")




cur.close()
conn.close()


rs： (2020371, None, 1608112661, 1608112748, 32, 1420212, 1420055, 0.05638062, 0.0011276123, 136.9286605, 35.1624512, 136.9288167, 35.1629419, '0102000020E610000002000000C9923996B71D614015996F33CB94414051CFCCDDB81D6140F5D2B947DB944140')
rs： (2020372, None, 1608112748, 1608112811, 32, 1420055, 1419919, 0.060645867, 0.0012129174, 136.9288167, 35.1629419, 136.9291233, 35.163414, '0102000020E61000000300000051CFCCDDB81D6140F5D2B947DB9441401863BBD6B91D61404337FB03E5944140736EC960BB1D6140B2DAFCBFEA944140')
rs： (2020460, None, 1608112629, 1608112588, 32, 1420081, 1420042, 0.030931138, 0.00061862275, 136.927948, 35.1623002, 136.928186, 35.1621014, '0102000020E6100000020000006F0C01C0B11D61402F91C140C69441405D5320B3B31D614062D119BDBF944140')
rs： (2020493, None, 1608112685, 1608112629, 32, 1420080, 1420081, 0.045348078, 0.00090696156, 136.9282819, 35.1626032, 136.927948, 35.1623002, '0102000020E610000002000000003F3E7CB41D6140CB1C812ED09441406F0C01C0B11D61402F91C140C6944140')
rs： (2020653, None, 160

In [24]:
$.ajax({
    url: '/getStrokes',
    type: 'POST',
    contentType: 'application/json',
    data: JSON.stringify({points: points}),
    success: function(response) {
      var strokes = response.stroke_wkts;
      strokes.forEach(function(wkt) {
        // "LINESTRING(lon lat,lon lat, ...)"をパース
        var coords = wkt.match(/\((.*)\)/)[1].split(',').map(function(pt) {
          var ll = pt.trim().split(' ').map(Number);
          return [ll[1], ll[0]]; // [lat, lng] 順に
        });
        var poly = L.polyline(coords, {color: 'purple'});
        poly.addTo(map);
      });
    }
});

SyntaxError: invalid syntax (1441037551.py, line 1)

In [23]:
<div id="map" style="height:600px"></div>
<script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
<script>
var map = L.map('map').setView([35.1625,136.929], 15);
L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png').addTo(map);

// fetchでAPIから全件取得
fetch('/get_strokes')
.then(response => response.json())
.then(data => {
    // dataは[[id, link_ids, stroke_length, stroke_clazz, WKT], ...]の配列
    for(var i=0;i<data.length;i++){
        var wkt = data[i][4]; // ST_AsText(arc_series)
        // WKTを配列に直訳（MULTILINESTRING((x1 y1, x2 y2),...))前提
        // ライブラリ(Wicket.js等)無しで超単純に描く → 正規表現で素分解
        var matches = wkt.match(/\(\((.+)\)\)/);
        if(matches){
            var lines = matches[1].split('),(');
            for(var j=0;j<lines.length;j++){
                var points = lines[j].split(',').map(function(pt){
                    var xy = pt.trim().split(' ');
                    return [parseFloat(xy[1]),parseFloat(xy[0])]; // 緯度, 経度
                });
                L.polyline(points, {color:'red'}).addTo(map);
            }
        }
    }
});
</script>

SyntaxError: invalid character '（' (U+FF08) (4050650524.py, line 14)